# Paris

This notebook illustrates the hierarchical clustering of graphs by the [Paris algorithm](https://scikit-network.readthedocs.io/en/latest/reference/hierarchy.html).

In [1]:
from IPython.display import SVG

In [2]:
import numpy as np

In [3]:
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.hierarchy import Paris, cut_straight, dasgupta_score, tree_sampling_divergence
from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph, svg_dendrogram

## Graphs

In [4]:
graph = karate_club(metadata=True)
adjacency = graph.adjacency
position = graph.position

In [5]:
# hierarchical clustering
paris = Paris()
dendrogram = paris.fit_transform(adjacency)

In [6]:
image = svg_dendrogram(dendrogram)
SVG(image)

In [8]:
# cuts of the dendrogram
labels = cut_straight(dendrogram)
print(labels)

[1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [9]:
n_clusters = 4
labels, dendrogram_aggregate = cut_straight(dendrogram, n_clusters, return_dendrogram=True)
print(labels)

[0 0 0 0 3 3 3 0 1 0 3 0 0 0 1 1 3 0 1 0 1 0 1 2 2 2 2 2 2 2 1 2 1 1]


In [10]:
_, counts = np.unique(labels, return_counts=True)

In [11]:
# aggregate dendrogram
image = svg_dendrogram(dendrogram_aggregate, names=counts, rotate_names=False)
SVG(image)

In [13]:
# corresponding clustering
image = svg_graph(adjacency, position, labels=labels)
SVG(image)

In [15]:
# metrics
dasgupta_score(adjacency, dendrogram)

0.6655354449472097

## Directed graphs

In [17]:
graph = painters(metadata=True)
adjacency = graph.adjacency
position = graph.position
names = graph.names

In [18]:
# hierarchical clustering
paris = Paris()
dendrogram = paris.fit_transform(adjacency)

In [19]:
image = svg_dendrogram(dendrogram, names, n_clusters=3, rotate=True)
SVG(image)

In [21]:
# cut with 3 clusters
labels = cut_straight(dendrogram, n_clusters = 3)
print(labels)

[0 0 1 0 1 1 2 0 0 1 0 0 0 2]


In [22]:
image = svg_digraph(adjacency, position, names=names, labels=labels)
SVG(image)

In [24]:
# metrics
dasgupta_score(adjacency, dendrogram)

0.5842857142857143

## Bipartite graphs

In [26]:
graph = movie_actor(metadata=True)
biadjacency = graph.biadjacency
names_row = graph.names_row
names_col = graph.names_col

In [27]:
# hierarchical clustering
paris = Paris()
paris.fit(biadjacency)
dendrogram_row = paris.dendrogram_row_
dendrogram_col = paris.dendrogram_col_
dendrogram_full = paris.dendrogram_full_

In [28]:
image = svg_dendrogram(dendrogram_row, names_row, n_clusters=4, rotate=True)
SVG(image)

In [30]:
image = svg_dendrogram(dendrogram_col, names_col, n_clusters=4, rotate=True)
SVG(image)

In [32]:
# cuts
labels = cut_straight(dendrogram_full, n_clusters = 4)
n_row = biadjacency.shape[0]
labels_row = labels[:n_row]
labels_col = labels[n_row:]

In [33]:
image = svg_bigraph(biadjacency, names_row, names_col, labels_row, labels_col)
SVG(image)